
## Applied Data Science Capstone Course - Week 3 Assignment - Part 2


Assignment Instructions:

Use the Geocoder Python package (https://geocoder.readthedocs.io/index.html) or a csv file that has the geographical coordinates of each postal code (http://cocl.us/Geospatial_data) to retrieve the Latitude and Longtitude of a given postal code.
Add these 2 columns to the dataframe from Part 1 of the Week 3 Assignment.


In [1]:
# using csv file as the source

# import pandas for work with dataframes
import pandas as pd

# read the source csv to pandas dataframe
df_geo = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Repeat steps from Week 3 Assignment - Part 1 to build the dataframe of Toronto neighborhoods

In [2]:
# import dependencies
import requests
from bs4 import BeautifulSoup
import lxml
import numpy as np

In [3]:
# get the web page using requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

# check the status (200 = success)
page.status_code

200

In [4]:
# parse the content of the page using BeautifulSoup and lxml parser
soup = BeautifulSoup(page.content, 'lxml')

# find the table that contains neighborhoods by postal code area
# note: when using the browser page inspector, it is evident 
# that the table with the required data is the first table of class "wikitable sortable" on the web page

table = soup.find('table', class_="wikitable sortable")


In [5]:
# clean up "Not assigned" values
clean_data = []

for row in table.find_all('tr'):
    cells = row.find_all('td')
    l=[]
    
    if len(cells) == 0:
        # do nothing
        pass
    else:
        PostalCode = cells[0].text
        Borough = cells[1].text
        Neighborhood = (cells[2].text).rstrip() #remove any whitespace / newline
    
        if Borough == "Not assigned":
            # skip records with Borough = "Not assigned"
            pass
        else:
            if Neighborhood == "Not assigned":
                # assign Neighborhood value from Borough
                Neighborhood = Borough
                
            # assemble clean values in a list    
            l.append(PostalCode)
            l.append(Borough)
            l.append(Neighborhood)

            clean_data.append(l)                

# create pandas dataframe form clean data
df = pd.DataFrame(clean_data, columns = ['PostalCode', 'Borough', 'Neighborhood'])
df.head()

        

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [6]:
# group Neighborhoods within the same Borough and PostalCode into a list 
group = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)

# convert list of Neighborhoods to String with values separated by ", "
group = group.str.join(", ")

# save group as dataframe
df_grouped = pd.DataFrame(group)

# reset index columns in the dataframe
df_grouped.reset_index(inplace=True)

df_grouped.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_grouped.shape

(103, 3)

In [8]:
# merge the two dataframes together
df_merged = pd.merge(df_grouped, df_geo, left_on = 'PostalCode', right_on = 'Postal Code')
df_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [9]:
# remove redundant Postal Code column
df_merged.drop(columns = 'Postal Code', inplace = True)
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
df_merged.shape

(103, 5)